# SQL tuning



Khi thực hiện các câu lệnh SQL ta cần lưu ý cách thức thực hiện các câu lệnh để đảm bảo tối ưu hóa về mặt hiệu năng & gia tăng tốc độ truy vấn. Chương này tổng hợp các mẹo giúp tăng tốc độ truy vấn SQL.

<center><font size=4>&sect;</font></center>

Ưu tiên sử dụng `JOIN` trước khi sử dụng các nhóm câu lệnh khác

<center><font size=4>&sect;</font></center>


Không nên dùng `select *`, sẽ tốn tài nguyên, dùng cột nào nên select ra cột đó


```sql
-- Dont
select * from customer

-- DO
select cif, cus_class from customer
```

<center><font size=4>&sect;</font></center>

Hạn chế dùng `distinct` nếu không thực sự cần thiết. Khi dùng `group by` rồi thì không cần distinct


```sql
-- Dont
select 
  distinct(branch), count(*) as no
from tbl_branch
group by branch

-- Do
select 
  branch, count(*) as no
from tbl_branch
group by branch
```

<center><font size=4>&sect;</font></center>

Khi dùng các kiểu điều kiện giá trị tồn tại/không tồn tại trong bảng khác, nên lưu ý sử dụng join hoặc `exists`


```sql
-- Create tables
use draft;

create table t1 (
	id int
	, prod1 varchar(8)
)

create table t2 (
	id int
	, prod1 varchar(8)
)

insert into t1 (id, prod1)
values
	( 1, 'A')
	,(2, 'B')
	,(3, 'C')

insert into t2 (id, prod1)
values
	( 1, 'A')
	,(1, 'E')
	,(5, 'F')

select * from t1
select * from t2

-- TH1: Select các giá trị có nằm trong bảng khác
-- Không nên dùng
select * from t2 A
where A.id in (select id from t1)

-- Nên dùng
select * from t2 A
where exists (
	select id from t1 B 
	where A.id = B.id
) 

```

```sql
-- TH2: Select các giá trị không nằm trong bảng khác
-- Không nên dùng - kiểu 1

select * from t2 A
where A.id not in (select id from t1)

-- Không nền dùng - kiểu 2
select * from t2 A
where not exists (
	select id from t1 B 
	where A.id = B.id
)  

-- Nên dùng
select A.* 
from t2 A
left join t1 B 
on A.id = B.id
where B.id is null

```

<center><font size=4>&sect;</font></center>

Khi `update`, nên dùng `update` kết hợp `inner join`


```sql
-- DONT
update t2
set prod1 = B.prod1
from t2 A, t1 B
where A.id = B.id

-- DO
update t2
set prod1 = B.prod1
from t2 A 
inner join t1 B
on A.id = B.id

```

<center><font size=4>&sect;</font></center>

Khi dùng `UPDATE`, lưu ý phải viết tường minh, tránh dùng cách viết thiếu tường minh như dưới đây


```sql
-- Dont
UPDATE A 
SET A.DIV= NULL
FROM (SELECT * FROM #TTR_TXN WHERE SRC_TBL='FT_HIST' AND DIV NOT IN ('SMEs','CIB','CMB')) A 

-- Do
Update #TTR_TXN
set DIV = NULL
where 
    SRC_TBL='FT_HIST' 
    AND DIV NOT IN ('SMEs','CIB','CMB')

```

<center><font size=4>&sect;</font></center>

Khi join hai bảng, cần phải lọc các điều kiện ra trước để bảng gọn hơn trước khi `join`, hạn chế join trước rồi mới `filter`.
Việc tạo bảng nhỏ trước khi join sẽ giúp tăng performance lên rất nhiều


```sql
--Dont
select * from 
	(
		select A.*
			, B.prod1 as prod2 
		from t1 A
		left join t2 B
		on A.id = B.id
	) tbl
where tbl.prod1 in ('A', 'C')

--Do
select A.*
	, B.prod1 as prod2 
from (
		select * 
		from t1
		where prod1 in ('A', 'C')
	) A
left join t2 B
on A.id = B.id

```

<center><font size=4>&sect;</font></center>

Ưu tiên sử dụng câu lệnh theo mức độ ưu tiên: `IN >> OR`, `EXISTS >> IN`, `BETWEEN >> IN`

<center><font size=4>&sect;</font></center>

Khi truy vấn, phải đảm bảo điều kiệu `sargable` (search argument able):

- Tránh sử dụng `function` trong điều kiện `where` do mất hiệu năng hơn và SQL sẽ không sử dụng được thuộc tính index
- Các cột nào là index nên đưa vào trước để filter
- Khi dùng Wild Card (dạng `%`), cần lưu ý hạn chế dùng like khi bắt đầu với `%` để tăng hiệu năng


```sql
--VD 1
-- câu lệnh 1 (non-sargable)
--Dont
SELECT * FROM Sales.Individual
WHERE CustomerID+2 = 11002

--DO
SELECT * FROM Sales.Individual
WHERE CustomerID = 11000


-- VD2
-- Dont
SELECT member_number, first_name, last_name
FROM members
WHERE DATEDIFF(yy,datofbirth,GETDATE()) > 21 

-- Do
SELECT member_number, first_name, last_name
FROM members
WHERE dateofbirth < DATEADD(yy,-21,GETDATE()) 

-- VD3
-- Dont
select * from customer where cus_name like '%HOANG%'

-- DO
select * from customer where cus_name like 'HOANG%'

```

<center><font size=4>&sect;</font></center>

- Khi dùng IN, có thể thay thế bằng UNION ALL


```sql
-- Câu lệnh 1: 242 ms 
set statistics time on 
select * from [CUSTOMER - M74 - BICDATA] where cus_name like 'VO%' or cus_name like '%PHAN%'

-- Câu lệnh 2: 172 ms
set statistics time on 
select * from [CUSTOMER - M74 - BICDATA] where cus_name like 'VO%'
union all
select * from [CUSTOMER - M74 - BICDATA] where cus_name like '%PHAN%'

```

<center><font size=4>&sect;</font></center>

Để xác định sự tồn tại của bản ghi, nên dùng `if exists`, không nên dùng `count(*)`


```sql
-- Dont
IF (SELECT COUNT(*) FROM table_name WHERE column_name = 'xxx')

-- Do
IF exists (SELECT * FROM table_name WHERE column_name = 'xxx')

```

<center><font size=4>&sect;</font></center>

Khi dùng hàm tính toán tổng hợp, nên đánh giá logic SQL sẽ phải đi như thế nào và nên dùng bảng dẫn (`derived table`) - bảng ở sau mệnh đề from để tính toán 


```sql
-- Dont
select min(trans) 
from hoang
where group_ in (
	select group_ from tbl_A
	where trans >= 15
)

-- Do
select min(A.trans) 
from (
	select group_, trans 
	from tbl_A
	where trans >= 15
) A

-- Dùng tương đương
with A as (
	select group_, trans 
	from hoang
	where trans >= 15
) 

```

<center><font size=4>&sect;</font></center>

Khi insert vào một bảng lớn, tránh dùng `union all` trước mà nên insert từng đoạn


```sql
-- Không nên dùng
with tbl_A as (
  select * from tbl1
  union all
  select * from tbl2
)
insert into #temp
select * from tbl_A

-- Nên dùng
insert into #temp
select * from tbl1

insert into #temp
select * from tbl2

```

<center><font size=4>&sect;</font></center>

Tránh sử dụng các hàm chuyển đổi kiểu dữ liệu trong mệnh đề WHERE. Ví dụ: `cast as`, `convert`.

<center><font size=4>&sect;</font></center>

Nếu nhận thấy rằng SQL Server sử dụng Table Scan thay vì INDEX SEEK trong câu truy vấn có `IN` hay `OR`, ngay cả cột tìm kiếm đó đã tạo index, ta có thể sử dụng thuộc tính `Index hint` để bắt buộc SQL tối ưu hoá truy vấn sử dụng `index`, như ví dụ sau:


```sql
--Dont
SELECT top 10 * from CUSTOMER.DBO.VPB_CUSTOMER  WHERE CIF in ('16258');

--Do
-- Câu truy vấn sau sẽ chạy nhanh hơn câu trước vì ép sử dụng index

SELECT top 10 * 
from CUSTOMER.DBO.VPB_CUSTOMER 
WITH(INDEX = IDX_CUSTOMER) WHERE CIF in ('16258');

```

<center><font size=4>&sect;</font></center>

Không dùng `EXISTS` hoặc `NOT EXISTS` trong câu lệnh `CASE WHEN`, tốc độ sẽ rất chậm. Cần `JOIN` trước khi sử dụng `CASE WHEN`


```sql
--DONT'
set STATISTICS  time on;

SELECT  CASE  WHEN MANAGER_ID = 'HANTT55' 
                  AND ACNT_CONTRACT_ID IN (SELECT ACNT_CONTRACT_ID from AUTO.DBO.PMC_RAW_RTC_202103) 
                  THEN N'Đóng bởi user của PMC'
              WHEN MANAGER_ID = 'HANTT55' AND ACNT_CONTRACT_ID 
                NOT IN (SELECT ACNT_CONTRACT_ID from AUTO.DBO.PMC_RAW_RTC_202103) 
                THEN N'Đóng bởi user của 247'
              WHEN MANAGER_ID <> 'HANTT55' THEN N'Đóng bởi user của Chi nhánh'
        ELSE    'Others' END MANAGER_ID
from    MONTHLY_NEWDATA.DBO.CREDITCARD_20210331 --> Tháng cần lấy dữ liệu
WHERE   CARD_ATTRITION = 1

--DO

set STATISTICS time on;
SELECT
CASE
    WHEN MANAGER_ID = 'HANTT55' then
        case
            when b.ACNT_CONTRACT_ID is not null tHEN N'Đóng bởi user của PMC'
            when b.ACNT_CONTRACT_ID is null THEN N'Đóng bởi user của 247'
        end
    WHEN MANAGER_ID <> 'HANTT55' THEN N'Đóng bởi user của Chi nhánh'
    ELSE 'Others' END MANAGER_ID
from MONTHLY_NEWDATA.DBO.CREDITCARD_20210331 a --> Tháng cần lấy dữ liệu
left join (
    select acnt_contract_id from AUTO.DBO.PMC_RAW_RTC_202103 group by acnt_contract_id) b
on a.ACNT_CONTRACT_ID = b.ACNT_CONTRACT_ID
WHERE A.CARD_ATTRITION = 1;

```

<center><font size=4>&sect;</font></center>

Sử dụng `Views` rất thuận tiện trong việc hạn chế người sử dụng xem dữ liệu nhưng về vấn đề hiêu năng. Tránh sử dụng các Views lồng nhau.


<center><font size=4>&sect;</font></center>


Trong SQL Server từ phiên bản 2000 cung cấp 1 kiểu dữ liệu mới gọi là `table` để lưu trữ tạm thời tập các records. 
Nếu có thể, ta hãy sử dụng biến table thay vì sử dụng bảng tạm.


```sql
declare @tbl_a table 
    (
        a varchar(10)
    )

```

<center><font size=4>&sect;</font></center>

Khi khai báo `datatype` column đúng với bản chất, tránh dư thừa tài nguyên. Ví dụ: cột `branch_id` có định dạng `VN12375` chỉ nên khai báo dạng `varchar(8)`, tránh khai báo `nvarchar(100)` do cột `branch_id` chỉ có tối đa 8 ký tự

<center><font size=4>&sect;</font></center>

Tránh sử dụng con trỏ (`SQL Server Cursors`) do các câu lệnh này thường sử dụng nhiều tài nguyên SQL Server, giảm hiệu năng của hệ thống

<center><font size=4>&sect;</font></center>

Khi dùng câu lệnh `UNION`, kết quả tương tự như `SELECT DISTINCT`. Do đó nếu biết chắc rằng không có 1 hàng nào trùng lắp được tạo ra từ kết quả của UNION thì ta nên sử dụng câu lệnh `UNION ALL`.

<center><font size=4>&sect;</font></center>

Tránh sử dụng các toán tử sau trong mệnh đề WHERE: "IS NULL", "<>", "!=", "!>", "!<", "NOT", "NOT EXISTS", "NOT IN", "NOT LIKE", and "LIKE '%abc'". 

<center><font size=4>&sect;</font></center>

Không dùng `having` thay thế cho where trong filter
